In [1]:
from src.melvin.stream import Stream
from src.melvin.Transcriber import Transcriber
from src.melvin.StreamTranscriber import StreamTranscriber

from src.eval.TimedStreamingTranscriber import TimedStreamingTranscriber
from src.eval.Dataset import Dataset
from src.eval.OutputHandler import OutputHandler

In [2]:
w = StreamTranscriber.for_gpu("large-v3-turbo", [0])
dataset = Dataset()

In [3]:
while True:
    id, X, y = next(dataset)
    if len(X)/32000 < 100:
        break
print(id)
print(len(X)/32000)

121-123859-0000
93.24


In [4]:
out = OutputHandler()
stream = Stream(w, 0, out)
transcriber = TimedStreamingTranscriber(stream, out, chunk_length_ms=50)
y_pred = await transcriber.transcribe(X)

Transcribing audio data with 2983680 bytes


In [5]:
out.final_words

[{'conf': 0.93457, 'start': 0.24, 'end': 0.82, 'word': 'You'},
 {'conf': 0.999512, 'start': 0.82, 'end': 1.14, 'word': 'are'},
 {'conf': 0.991211, 'start': 1.16, 'end': 1.46, 'word': 'my'},
 {'conf': 0.984863, 'start': 1.46, 'end': 1.82, 'word': 'all'},
 {'conf': 0.953125, 'start': 1.82, 'end': 1.98, 'word': 'the'},
 {'conf': 1.0, 'start': 1.98, 'end': 2.4, 'word': 'world'},
 {'conf': 0.368896, 'start': 2.4, 'end': 3.44, 'word': 'and'},
 {'conf': 0.808105, 'start': 3.44, 'end': 3.66, 'word': 'i'},
 {'conf': 0.999023, 'start': 3.66, 'end': 3.96, 'word': 'must'},
 {'conf': 1.0, 'start': 3.96, 'end': 4.48, 'word': 'strive'},
 {'conf': 0.998535, 'start': 4.48, 'end': 5.24, 'word': 'to'},
 {'conf': 0.999512, 'start': 5.24, 'end': 5.5, 'word': 'know'},
 {'conf': 0.999512, 'start': 5.5, 'end': 5.76, 'word': 'my'},
 {'conf': 0.998047, 'start': 5.76, 'end': 6.42, 'word': 'shames'},
 {'conf': 0.999512, 'start': 6.42, 'end': 6.74, 'word': 'and'},
 {'conf': 0.999756, 'start': 6.74, 'end': 7.26, 'w

In [6]:
for m in out.partial_predictions:
    print(f"Window {m['window'][0]:.2f} - {m['window'][1]:.2f} observed at {m['observation_time']:.4f} :  { m['result']['text']}")

Window 0.00 - 1.00 observed at 1.7205 :  You are
Window 0.00 - 2.00 observed at 2.3556 :  You are my old.
Window 0.00 - 3.00 observed at 3.3624 :  you are my all the world
Window 0.00 - 4.00 observed at 4.3813 :  you are my all the world and i must
Window 0.00 - 5.00 observed at 5.3748 :  you are my all the world and i must strive
Window 0.00 - 6.00 observed at 6.3967 :  strive to know my shame.
Window 0.00 - 7.00 observed at 7.4305 :  strive to know my shames and praise.
Window 0.00 - 8.00 observed at 8.4171 :  shames and praises from your tongue.
Window 0.00 - 9.00 observed at 9.4163 :  shames and praises from your tongue.
Window 0.00 - 10.00 observed at 10.4460 :  tongue.
Window 0.00 - 11.00 observed at 11.4336 :  tongue none else to me nor
Window 0.00 - 12.00 observed at 12.4473 :  none else to me nor i to none alone
Window 0.00 - 12.05 observed at 12.8683 :  none else to me nor i to none
Window 0.00 - 12.45 observed at 13.3101 :  alive
Window 0.00 - 12.90 observed at 13.7451 :  al

In [7]:
from pydub.utils import re

def __norm_word(word) -> str:
    text = word.lower()
    # Remove non-alphabetic characters using regular expression
    text = re.sub(r"[^a-z]", "", text)
    return text.lower().strip().strip(".,?!")

def remove_duplicates(words):
    i = 1
    new_words = words
    while i < len(new_words):
        if (
            __norm_word(new_words[i]["word"]) == __norm_word(new_words[i-1]["word"])
            and new_words[i]["start"] < new_words[i-1]["end"]
        ):
            new_words.pop(i)
        else:
            i += 1
    return new_words

In [8]:
y_gold = []
for result in stream.final_transcriptions:
    y_gold += result["result"]

y_gold = remove_duplicates(y_gold)
y_gold_text = " ".join([w["word"] for w in y_gold])
print(y_gold_text)
y_gold

You are my all the world and i must strive to know my shames and praises from your tongue. none else to me nor i to none alive that my steeled sense or changes right or wrong. nd I must strive I must strive tis flattery in my seeing, and my great mind most kingly drink it up mine mine eye well knows what with his gust is right or wrong If it be poisoned, tis the lesser sin That mine eye loves it and doth first begin nd to his palate doth prepare the cup, Creep in twixt vows and change decrees of kings Tan sacred beauty Blunt the sharpest intense intents, divert strong minds to the course of altering things, alas, why fearing of time's tyranny? might i not then say now i love you best. when I was certain or in certainty? crowning the present, doubting of the rest? Love is a babe. then might i not say so to give full growth to that which still doth grow so I return rebuked to my content and gain by ill thrice more than I have more than I have spent.


[{'conf': 0.93457, 'start': 0.24, 'end': 0.82, 'word': 'You'},
 {'conf': 0.999512, 'start': 0.82, 'end': 1.14, 'word': 'are'},
 {'conf': 0.991211, 'start': 1.16, 'end': 1.46, 'word': 'my'},
 {'conf': 0.984863, 'start': 1.46, 'end': 1.82, 'word': 'all'},
 {'conf': 0.953125, 'start': 1.82, 'end': 1.98, 'word': 'the'},
 {'conf': 1.0, 'start': 1.98, 'end': 2.4, 'word': 'world'},
 {'conf': 0.368896, 'start': 2.4, 'end': 3.44, 'word': 'and'},
 {'conf': 0.808105, 'start': 3.44, 'end': 3.66, 'word': 'i'},
 {'conf': 0.999023, 'start': 3.66, 'end': 3.96, 'word': 'must'},
 {'conf': 1.0, 'start': 3.96, 'end': 4.48, 'word': 'strive'},
 {'conf': 0.998535, 'start': 4.48, 'end': 5.24, 'word': 'to'},
 {'conf': 0.999512, 'start': 5.24, 'end': 5.5, 'word': 'know'},
 {'conf': 0.999512, 'start': 5.5, 'end': 5.76, 'word': 'my'},
 {'conf': 0.998047, 'start': 5.76, 'end': 6.42, 'word': 'shames'},
 {'conf': 0.999512, 'start': 6.42, 'end': 6.74, 'word': 'and'},
 {'conf': 0.999756, 'start': 6.74, 'end': 7.26, 'w